# Imports

In [ ]:
import os
import numpy as np

import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from pydub import AudioSegment
import wave
import math

# Load data

In [ ]:
model_name = 'flowtron'

In [ ]:
main_directory = os.getcwd() + "\\..\\datasets\\{}\\\wavs\\".format(model_name)
audio_lengths = []

files = [f for f in os.listdir(main_directory) if os.path.isfile(os.path.join(main_directory, f))]
for f in files:
    audio = AudioSegment.from_file(main_directory + f)
    audio_lengths.append(audio.duration_seconds)

In [ ]:
data = pd.DataFrame(audio_lengths, columns=["Length"])
data.head()

In [ ]:
data["Length"].describe()

# Dataset length (s)

In [ ]:
data_audio = data["Length"].sum()
data_audio_formatted = '{} hours {} minutes {} seconds'.format(
    math.floor(data_audio / 3600), 
    math.floor((data_audio % 3600) / 60),
    round((data_audio % 3600) % 60, 1))
print("Total dataset length:", data_audio_formatted)

In [ ]:
plt.figure(figsize=(10, 6))
sns.set_style("darkgrid")
sns.histplot(data["Length"], bins=20, kde=False)
plt.xlabel("Audio length (seconds)")
plt.ylabel("Number of files")
plt.title("Histogram of audio file lengths in {} dataset".format(model_name))

# Sample file characteristics

In [ ]:
file_name = main_directory + "DIA_ADDON_1013_BANDIT_FROMCAVALORN_15_00.wav"
audio_file = wave.open(file_name, "r")
signal = audio_file.readframes(-1)
signal = np.frombuffer(signal, dtype=np.int16)
plt.figure(figsize=(14, 6))
sns.lineplot(data=signal)
plt.title("Sample audio plot")